# First steps to creating a Django dashboard

I generate my own plots with R or Python, typically with ggplot or matplotlib, although I've sometimes used other libraries or even Javascript and D3.

Recently, I needed to create an internal dashboard for an organization to track various metrics. I looked into options like Tableau and Plotly, but realized I didn't like 1) paying for something I could do easily, 2) adding additional data formatting steps, or 3) having all the bells and whistles. Instead, I just modified my existing visualization workflow to serve plots via Django.

Basically, we're going to have a URL linked to a view that returns a figure, and a template with an image link to that URL. Yeah, this isn't earth-shattering, but it's so much easier to do this when you're already creating visualizations and working within a Django project, and I'd like to remember how to do this in the future.

## Generate the figure

First, I have an analytics module in my Django project, and I added a visualizations script to create figures. In that script, I need to import matplotlib and configure the backend:

```
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
```

Within that script, I need to create a function to generate a figure:

```
def create_figure():
    {magic}
    return figure
```

## Generate the dashboard template

Second, I create a template at main/templates/dashboard.html that displays visualizations, although it can certainly be cleaned up with a framework like Bootstrap:

```
{% extends "base.html" %}

{% block content %}

<h1>Sample visualization</h1>
<br>

<img src="{% url 'DashboardVisualizations' %}">

{% endblock %}
```


## Create view to handle requests

Third, we need to create a view for the dashboard and a view for the dashboard visualizations. The former returns the webpage, while the later returns the figure. Thus, we create a script called main/views/dashboards.py with this content:

```
from django.http import HttpResponse
from matplotlib.backends.backend_agg import FigureCanvasAgg

import main.analytics.visualizations


class Dashboard(TemplateView):

    template_name = 'main/templates/dashboard.html'  # This was our template filename

    def get_context_data(self, **kwargs):
        # Context can be used to modify dashboard or figure content
        context = {'sample_key': 'sample_context'}
        return context


class DashboardVisualizations(View):

    def get(self, request, *args, **kwargs):
        # The following function is from our visualization script
        figure = visualizations.create_figure()  
        canvas = FigureCanvasAgg(figure)
        response = HttpResponse(content_type='image/png')
        canvas.print_png(response)
        return response
```

## Configure URLs to handle requests

Finally, we need to create a URL for both the dashboard and the visualizations so that requests are routed to the correct views.

```
# We import the views from our view script
from main.views.dashboards import Dashboard, DashBoardVisualizations

urlpatterns = [
    ...
    url(r'dashboard/', Dashboard.as_view()),
    url(r'dashboard/sample_plot/', DashboardVisualizations.as_view()),
    ...
    ]
```

Now, you just need to navigate to projectname.com/dashboard to see the figures!